In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['label'] = iris.target

In [ ]:
y = df['label']
X = df.drop(['label'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, stratify=y)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(4, 16)
        self.hidden_layer1  = nn.Linear(16, 32)
        self.output_layer   = nn.Linear(32, 3)
        self.relu = nn.ReLU()    
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device = {device}')

model = NeuralNetwork().to(device)

device = cpu


In [ ]:
learning_rate = 0.001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
def train_loop(train_loader, model, loss_fn, optimizer):
    size = len(train_loader.dataset)

    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)

        # 손실 계산
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss, current = loss.item(), batch * len(X)
        print(f'loss: {loss:>7f}  [{current:>5d}]/{size:5d}')


In [ ]:
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")

In [ ]:
epochs = 10

for i in range(epochs) :
    print(f"Epoch {i+1} \n------------------------")
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model, loss)
print("Done!")

Epoch 1 
------------------------
loss: 1.107460  [    0]/  112
loss: 1.142883  [   10]/  112
loss: 1.092416  [   20]/  112
loss: 1.130637  [   30]/  112
loss: 1.225811  [   40]/  112
loss: 1.079539  [   50]/  112
loss: 1.011783  [   60]/  112
loss: 1.009363  [   70]/  112
loss: 1.052150  [   80]/  112
loss: 1.062375  [   90]/  112
loss: 1.061682  [  100]/  112
loss: 1.006883  [   22]/  112
Test Error: 
 Accuracy: 34.2%, Avg loss: 1.026808

Epoch 2 
------------------------
loss: 1.038208  [    0]/  112
loss: 1.098868  [   10]/  112
loss: 0.988461  [   20]/  112
loss: 1.037010  [   30]/  112
loss: 0.993803  [   40]/  112
loss: 1.002819  [   50]/  112
loss: 0.899770  [   60]/  112
loss: 1.027900  [   70]/  112
loss: 1.002770  [   80]/  112
loss: 0.969722  [   90]/  112
loss: 1.096071  [  100]/  112
loss: 1.106542  [   22]/  112
Test Error: 
 Accuracy: 34.2%, Avg loss: 0.976655

Epoch 3 
------------------------
loss: 1.057000  [    0]/  112
loss: 0.918456  [   10]/  112
loss: 0.984511  